In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings

llm = ChatOllama(model="gemma3:4b")
# embeddings = HuggingFaceEndpointEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")


In [ ]:
data = PyPDFLoader("D:\Projects2025\Chatbot2\Media\operating system.pdf").load()

spliter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=200)

docs =  spliter.split_documents(data)
docs

In [ ]:
from langchain_community.vectorstores import FAISS


vs = FAISS.from_documents(embedding=embeddings, documents=docs)

vs

In [ ]:
vs.similarity_search("What is an operating system")

In [ ]:
retriver = vs.as_retriever()

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "you are an helpful assistant help to resolve this query based on the {context}. if the query is not relevent say i don't know "),
        ("human", "{input}")
    ]
)

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


context_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag = create_retrieval_chain(retriver, context_chain)
rag

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E2FDB1A3C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="you are an helpful assistant help to resolve this query based on the {context}. if the query is not relevent say i don't know 

In [19]:
response = rag.invoke({"input": "What is an hululu?"})
response

{'input': 'What is an hululu?',
 'context': [Document(id='caf932a8-10f0-4e1d-bc3a-8737ec2627f4', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 77, 'page_label': '78'}, page_content='Explanation: None.\n10. The media file is streamed to the client but is only played and not stored \nby the client in ___________\na) progressive download\nb) regular download\nc) real time streaming\nd) virtual time streaming\nAnswer: c\nExplanation: None.\n11. Real time streaming is most useful for ___________\na) short video clips\nb) long video clips\nc) extremely short and low quality videos\nd) none of the mentioned\nAnswer: b\nExplanation: None.\nMore MCQs on Multimedia System – Compression:\n--------------------------------------------------------------------------------\n---------------------------------\n

In [20]:
response = rag.invoke({"input": "What is my name?"})
response

{'input': 'What is my name?',
 'context': [Document(id='b02c209e-c1de-47fd-8e69-a16d96dd372e', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 129, 'page_label': '130'}, page_content='8. The machine containing the files is the _______ and the machine wanting to \naccess the files is the ______\na) master, slave\nb) memory, user\nc) server, client\nd) none of the mentioned\nAnswer: c\nExplanation: None.\n9. Distributed naming services/Distributed information systems have been devised \nto _____________\na) provide information about all the systems\nb) provide unified access to the information needed for remote computing\nc) provide unique names to all systems in a network\nd) all of the mentioned\nAnswer: b\nExplanation: None.\n10. Domain name system provides _____________\na) host-name-to-networ

In [28]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("chat_history"),
    ]
)

In [30]:

h_ritriver = create_history_aware_retriever(llm, retriver, contextualize_q_prompt)
h_ritriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E2FDB1A3C0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Ta

In [31]:
stuff_doc = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(h_ritriver, stuff_doc)

In [33]:
question = "What is an operating system?"
response1=rag.invoke({"input":question})
response1["answer"]

'According to the provided information:\n\n**An operating system is a collection of programs that manages hardware resources.** \n\nIt also provides services to application programs and acts as an interface between the hardware and application programs.'

In [39]:
question = "To access the services of it, the interface is provided by the?"
response1=rag.invoke({"input":question})
response1["answer"]

'To access the services of the operating system, the interface is provided by the **system calls**.'

In [36]:
question = "Hello my name is Nafeesh?"
response1=rag.invoke({"input":question})
response1["answer"]

'Hello Nafeesh! It’s nice to meet you. 😊 \n\nHow are you doing today?'

In [38]:
question = "What is my name?"
response1=rag.invoke({"input":question})
response1["answer"]

"I do not know your name. As an AI, I have no memory of past conversations and don't retain personal information. \n\nTo help me understand who you are, you’ll need to tell me your name! 😊"

In [41]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question= "Hello my name is Nafeesh?"
response1=rag.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Hello Nafeesh! It’s nice to meet you. 😊 

How are you doing today?


In [42]:
question2="What is my name?"
response2=rag.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Hello Nafeesh! It’s nice to meet you. 😊 

Is there anything I can help you with today?
